<a href="https://colab.research.google.com/github/GitMishka/2015-2017_amzn_aaple_tsla_twtr_allocation_pract/blob/master/Reddit_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praw
import praw
from google.colab import drive
import pandas as pd
import datetime
import psycopg2 as ps


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [ ]:
# #database credentials
# host_name = 'my-db1.cwc1bi2tp4j7.us-east-2.rds.amazonaws.com'
# dbname = 'initial_db1'
# port = '5432'
# username = 'postgres' 
# password = 'Manonthemoon123'
# conn = None

# #establish a connection to db
# conn = connect_to_db(host_name, dbname, port, username, password)
# curr = conn.cursor()


In [ ]:
user_agent = "Searchbot_01"
reddit = praw.Reddit(username="Searchbot_01",
                     password ="aaaa1111",
                     client_id="Ai32qfXNqvGuMEvHFFMlAw",
                     client_secret="IG5XKjyUGkcG2cgXfBSwVvalMTxFRg",
                     user_agent=user_agent)

subreddit_name = "watchexchange"
subreddit = reddit.subreddit(subreddit_name)
#print(subreddit.display_name)


df = pd.DataFrame() # creating dataframe for displaying scraped data

# creating lists for storing scraped data
title=[]
upvotes=[]
post_id=[]
time_created = []
comments = []
url= []
#author = []
price = ['USD', 'PRICE','$']
upvote_ratio = []

In [ ]:
for submission in subreddit.top(limit=10):
    title.append(submission.title)
    upvotes.append(submission.score) #upvotes
    post_id.append(submission.id)
    #author.append(submission.author)
    time_created.append(datetime.datetime.utcfromtimestamp(submission.created))
    upvote_ratio.append(submission.upvote_ratio)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
df['post_id'] = post_id
df['title'] = title
df['upvotes'] = upvotes #upvotes
df['time_created'] = time_created
#df['author'] = author
df['upvote_ratio'] = upvote_ratio

In [ ]:
#df.set_index('post_id', inplace=True)
print(df.shape)
#df.head(10)
#print(df)
print(df.columns)

(10, 5)
Index(['post_id', 'title', 'upvotes', 'time_created', 'upvote_ratio'], dtype='object')


In [ ]:
try:
  df.to_csv('df1.csv')
  print('Success')
except:
  print('Failed')

Success


In [ ]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS watch_exchange (
                    post_id VARCHAR(10) PRIMARY KEY,
                    title VARCHAR(255),
                    upvotes INTEGER NOT NULL,
                    time_created DATE NOT NULL DEFAULT CURRENT_DATE,
                
                    upvote_ratio decimal NOT NULL           
            )""")
    curr.execute(create_table_command)

def insert_into_table(curr, post_id, title, upvotes, time_created, upvote_ratio):
  insert_into_exchange = ("""INSERT INTO watch_exchange (post_id, title,  upvotes, time_created,
                         upvote_ratio) 
                        VALUES (%s,%s,%s,%s,%s);""")
  row_to_insert = (post_id, title,  upvotes, time_created,  upvote_ratio)
  curr.execute(insert_into_exchange, row_to_insert)

def update_row(curr,title,post_id,upvotes,time_created,upvote_ratio):
  query = ("""UPDATE watch_exchange SET title = %s, upvotes = %s, time_created = %s,  upvote_ratio = %s where post_id = %s;""")
  vars_to_update = (title,upvotes,time_created,upvote_ratio, post_id)
  curr.execute(query, vars_to_update)
  
def check_if_post_exists(curr, post_id):
  query = ("""SELECT post_id FROM watch_exchange where post_id = %s""")
  curr.execute(query, (post_id,))
  return curr.fetchone() is not None

def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE watch_exchange""")

    curr.execute(truncate_table)
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['post_id'], row['title'],  row['upvotes'], row['time_created']
                          , row['upvote_ratio'])
        
def update_db(curr,df):
  tmp_df = pd.DataFrame(columns = ['post_id', 'title', 'upvotes', 'time_created',  'upvote_ratio'])
  for i, row in df.iterrows():
    if check_if_post_exists(curr, row['post_id']):
      update_row(curr,row['post_id'], row['title'],row['upvotes'],row['time_created'],row['upvote_ratio'])
    else:
      tmp_df = tmp_df.append(row)
  return tmp_df

In [ ]:
#database credentials
host_name = 'my-db1.cwc1bi2tp4j7.us-east-2.rds.amazonaws.com'
dbname = 'initial_db1'
port = '5432'
username = 'postgres' 
password = 'Manonthemoon123'
conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [ ]:
create_table(curr)
conn.commit()

In [ ]:
new_post_df = update_db(curr,df)
conn.commit()

In [ ]:
append_from_df_to_db(curr, new_post_df)
conn.commit()

In [ ]:
curr.execute("SELECT * FROM watch_exchange")
print(curr.fetchall())

[('11es1qh', '[GIVEAWAY] Win a Casio Mini-Collection by Simply Upvoting and Commenting!', 15695, datetime.date(2023, 3, 1), Decimal('0.87')), ('ztz3nj', '[GIVEAWAY] Win a Watch Roll of your by simply upvoting and commenting!', 15411, datetime.date(2022, 12, 24), Decimal('0.83')), ('i2zk7s', "[GIVEAWAY] Unworn Retro Casio - Will match upvotes/donations to Parkinson's Foundation", 3084, datetime.date(2020, 8, 3), Decimal('0.99')), ('l52wyo', '[WTS] Custom 40mm build with laser treated titanium Great Wave Off Kanagawa Dial', 1676, datetime.date(2021, 1, 26), Decimal('0.99')), ('w7dwic', '[WTS] Complete TT Sub - 1999 - Rare Swiss Only Dial, Box, and Papers, Extras -Unpolished- Discounting price with each upvote.', 1622, datetime.date(2022, 7, 25), Decimal('0.95')), ('fl9z9g', '[WTS] Charmin + Purell + Rolex Oyster Perpetual 114200 34mm Grape Dial Full Set OP34', 1554, datetime.date(2020, 3, 19), Decimal('0.95')), ('gmy1lj', '[WTS] Rolex Yacht-Master, Ref. 16622 with A serial', 1468, dateti

In [ ]:
#insert_into_table(curr,row['title'],row['post_id'],row['upvote',row['time_created'],row['author'],row['upvote_ratio'])

# New Section